In [60]:
import tensorflow as tf
import csv
import numpy as np

In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
embedding_dim = 100
#max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=5550
test_portion=.1

corpus = []

In [63]:
num_sentences = 0

with open("sms scam ready dataset.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        list_item=[]
        list_item.append(row[2])
        list_item.append(row[0])
        num_sentences = num_sentences + 1
        corpus.append(list_item)

In [64]:
print(num_sentences)
print(len(corpus))
print(corpus[54])

5572
5572
['SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV', '1']


In [76]:
sentences=[]
labels=[]
for x in range(training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])

print(sentences[0])
print(labels[0])
labels[0] = 0    
tokenizer = Tokenizer(num_words=9000, oov_token= oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
vocab_size=len(word_index)

print(vocab_size)

sequences = tokenizer.texts_to_sequences(sentences)
max_length = max([len(seq) for seq in sequences])
print("maximum length of sentence is ",max_length)

padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(padded)
split = int(test_portion * training_size)

test_sequences = padded[0:split]
training_sequences = padded[split:training_size]
test_labels = labels[0:split]
training_labels = labels[split:training_size]


print(len(padded))
print(len(training_sequences)+len(test_sequences))
print(len(training_labels)+len(test_labels))
print(labels)

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ï»¿0
8991
maximum length of sentence is  189
[[  51  471 4433 ...    0    0    0]
 [  47  336 1497 ...    0    0    0]
 [  49  488    9 ...    0    0    0]
 ...
 [ 137   14 1624 ...    0    0    0]
 [  39    2   61 ...    0    0    0]
 [   4   55  497 ...    0    0    0]]
5550
5550
5550
[0, '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '

In [77]:
print(vocab_size)
print(word_index['a'])

8991
5


In [78]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 189, 100)          899200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 189, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 185, 64)           32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 46, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 964,353
Trainable params: 964,353
Non-trainable params: 0
_________________________________________________________________


In [80]:
num_epochs = 25
training_sequences = np.array(training_sequences)
training_labels = np.array(training_labels)
test_sequences = np.array(test_sequences)
test_labels = np.array(test_labels)
history = model.fit(training_sequences, training_labels, epochs=num_epochs, validation_data=(test_sequences, test_labels), verbose= 1)

print("Training Complete")


Train on 4995 samples, validate on 555 samples
Epoch 1/25
4995/4995 [==============================] - 23s 5ms/sample - loss: 0.3929 - acc: 0.8669 - val_loss: 0.4170 - val_acc: 0.8559
Epoch 2/25
4995/4995 [==============================] - 27s 5ms/sample - loss: 0.3927 - acc: 0.8669 - val_loss: 0.4127 - val_acc: 0.8559
Epoch 3/25
4995/4995 [==============================] - 27s 5ms/sample - loss: 0.3926 - acc: 0.8669 - val_loss: 0.4127 - val_acc: 0.8559
Epoch 4/25
4995/4995 [==============================] - 28s 6ms/sample - loss: 0.3931 - acc: 0.8669 - val_loss: 0.4132 - val_acc: 0.8559
Epoch 5/25
4995/4995 [==============================] - 27s 5ms/sample - loss: 0.3928 - acc: 0.8669 - val_loss: 0.4124 - val_acc: 0.8559
Epoch 6/25
4995/4995 [==============================] - 28s 6ms/sample - loss: 0.3928 - acc: 0.8669 - val_loss: 0.4124 - val_acc: 0.8559
Epoch 7/25
4995/4995 [==============================] - 27s 5ms/sample - loss: 0.3926 - acc: 0.8669 - val_loss: 0.4124 - val_acc: 0